# 5. SQLAlchemy – Some Commonly Asked Questions
http://pythoncentral.io/sqlalchemy-faqs/

In [ ]:
%reset

#### SQLAlchemy Schema Reflection / Introspection

Instead of creating a schema automatically from the SQLAlchemy, as what's shown in the previous articles using Base.metadata.create_all(engine), we can also instruct a Table object to load information about itself from the corresponding database schema object already existing within the database.

Let's create an example sqlite3 database with one table person that stores one record:

In [ ]:
import sqlite3
 
conn = sqlite3.connect("example.db")
c = conn.cursor()
c.execute('''
          CREATE TABLE IF NOT EXISTS person  (name text, email text)
          ''')
c.execute("INSERT INTO person VALUES ('john', 'john@example.com')")
c.close()

Now we can reflect the structure of table person using the arguments **autoload** and **autoload_with** in the Table constructor.

In [ ]:
from sqlalchemy import create_engine, MetaData, Table

engine = create_engine('sqlite:///example.db')
meta = MetaData(bind=engine)
person = Table("person", meta, autoload=True, autoload_with=engine)
person

In [ ]:
[c.name for c in person.columns]

We can also reflect all tables in the database using the MetaData.reflect method.

In [ ]:
meta = MetaData()
meta.reflect(bind=engine)
person = meta.tables['person']
person

Albeit very powerful, reflection does have its limitations. It's important to remember reflection constructs Table metadata using only information available in the relational database. Naturally, such a process cannot restore aspects of a schema that are not actually stored in the database. The aspects that are not available include but not limited to:

1. Client side defaults, Python functions or SQL expressions defined using the default keyword of the Column constructor.
2. Column information, defined in the Column.info dictionary.
3. The value of the .quote setting for Column or Table.
4. The association of a particular Sequence with a given Column.  

Recent improvements in SQLAlchemy allow structures like views, indexes and foreign key options to be reflected. Structures like CHECK constraints, table comments and triggers are not reflected.

#### Performance Overhead of SQLAlchemy

Since SQLAlchemy uses the **unit of work** pattern when synchronizing changes, i.e., **session.commit()**, to the database, it does more than just "inserts" data as in a raw SQL statement. It tracks changes made to a session's object and maintain an identity map for all the objects. It also performs a fair bit amount of bookkeeping and maintains the integrity of any CRUD operations. Overall, unit of work automates the task of persisting a complex object graph into a relational database without writing explicit procedural persistence code. Of course, such an advanced automation has a price.

Since SQLAlchemy's ORM is not designed to deal with bulk insertions, we can write an example to test its efficiency against raw SQL. Besides the ORM and raw SQL implementation of a bulk insertion test case, we also implement a version that uses SQLAlchemy's **Core system**. Since SQLAlchemy's Core is a thin layer of abstraction above the raw SQL, we expect it to achieve comparable level of performance to raw SQL.

In [ ]:
import time
import sqlite3

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.orm import scoped_session, sessionmaker

Base = declarative_base()
session = scoped_session(sessionmaker())

class User(Base):
    __tablename__ = "user"
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
 
 
def init_db(dbname='sqlite:///example1.db'):
    engine = create_engine(dbname, echo=False)
    session.remove()
    session.configure(bind=engine, autoflush=False, expire_on_commit=False)
    Base.metadata.drop_all(engine)
    Base.metadata.create_all(engine)
    return engine

def test_sqlalchemy_orm(number_of_records=100000):
    init_db()
    start = time.time()
    for i in range(number_of_records):
        user = User()
        user.name = 'NAME ' + str(i)
        session.add(user)
    session.commit()
    end = time.time()
    print ("SQLAlchemy ORM: Insert {0} records in {1} seconds".format(
        str(number_of_records), str(end - start)
    ))
 
 
def test_sqlalchemy_core(number_of_records=100000):
    engine = init_db()
    start = time.time()
    engine.execute(
        User.__table__.insert(),
        [{"name": "NAME " + str(i)} for i in range(number_of_records)]
    )
    end = time.time()
    print ("SQLAlchemy Core: Insert {0} records in {1} seconds".format(
        str(number_of_records), str(end - start)
    ))

def init_sqlite3(dbname="sqlite3.db"):
    conn = sqlite3.connect(dbname)
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS user")
    cursor.execute("CREATE TABLE user (id INTEGER NOT NULL, name VARCHAR(255), PRIMARY KEY(id))")
    conn.commit()
    return conn
 
def test_sqlite3(number_of_records=100000):
    conn = init_sqlite3()
    cursor = conn.cursor()
    start = time.time()
    for i in range(number_of_records):
        cursor.execute("INSERT INTO user (name) VALUES (?)", ("NAME " + str(i),))
    conn.commit()
    end = time.time()
    print ("sqlite3: Insert {0} records in {1} seconds".format(
        str(number_of_records), str(end - start)
    ))

def doIt():    
    test_sqlite3()
    test_sqlalchemy_core()
    test_sqlalchemy_orm()

if __name__ == "__main__":  
    doIt()

In the previous code, we compare the performance of bulk inserting 100000 user records into a sqlite3 database using raw SQL, SQLAlchemy's Core and SQLAlchemy's ORM.  

Notice that the Core and raw SQL achieved comparable insertion speed while the ORM is much slower than the other two.  
Although it looks like the ORM incurs a large performance overhead, keep in mind that the overhead becomes significant only when there is a large amount of data to be inserted.  
Since most web applications run small CRUD operations in one request-response cycle, **it's preferred to using the ORM instead of the Core due to the extra convenience and better maintainability.**

#### SQLAlchemy and database permissions

So far, our examples have been working well with sqlite3 databases, which do not have fine-grained **access control** such as user and permission management. What if we want to use SQLAlchemy with MySQL or PostgreSQL? What happens when the user connected to the database does not have enough permission to create tables, indexes, etc.? Will SQLAlchemy throw a database access exception?

Let's use an example to test the behaviour of SQLAlchemy's ORM when there is not enough permissions given to a user. First, we create a testing database "test_sqlalchemy" and a testing user "sqlalchemy".

#### SQLAlchemy's Schema Migration

There are at least two libraries available for performing SQLAlchemy migrations: **migrate** documentation link and **alembic** documentation link.

Since **alembic** was written by the author of SQLAlchemy and actively developed, we recommend you to use it instead of migrate. Not only does alembic allow you to manually write migration scripts, it also provides a way to auto-generate the scripts. We will further explore how to use alembic in another article.

#### SQLAlchemy's Support for Triggers

SQL **triggers** can be created using custom DDL constructs and hooked to SQLAlchemy's **events**. Although it's not a direct support for triggers, it's easy to implement and plug into any system. We will take a look at custom DDL and events in another article.

#### Tips and Summary
In this article, we answered a couple common questions regarding SQLAlchemy from a SQL database admin's point of view.  

Although SQLAlchemy defaults to create a database schema for you, **it also allows you to reflect on an existing schema and generates Table objects for you**.  

There's a performance overhead when using SQLAlchemy's ORM, but it's mostly obvious when performing bulk insertions, while most web applications perform relatively small CRUD operations.  

If your database user does not have enough permissions to perform certain actions on a table, SQLAlchemy will throw an exception that shows exactly why you cannot perform the actions.  

There are two migration libraries for SQLAlchemy and alembic is highly recommended.  

Although triggers are not directly supported, you can easily write them in raw SQL and hook them up using custom DDL and SQLAlchemy events.